In [26]:
#Celda para librerías
import sklearn as sk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
import json
import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos

app_train = pd.read_csv(r'/home/yeray/home-credit-default-risk/application_train.csv')
app_test=pd.read_csv(r'/home/yeray/home-credit-default-risk/application_test.csv')
app_test_pruebav1 = app_test.copy()
bureau_final = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/bureau_agg_final.csv')
bureau_balance__agg_def_v1 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/bureau_balance_agg_def.csv')
previous_application_agg = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/prev_app.csv')
pos_cash_agg = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/pos_cash_agg_final.csv')
installment_payment_agg = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/installment_payments_agg.csv')
credit_card_balance = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/credit_card_balance_agg.csv')
print(app_train.shape
      , bureau_final.shape
      , bureau_balance__agg_def_v1.shape
      , previous_application_agg.shape
      , pos_cash_agg.shape)


2.1.3
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS
(307511, 122) (305811, 404) (305811, 23) (338857, 361) (337252, 37)


In [14]:
# Verificar los tipos de datos
print(app_train['SK_ID_CURR'].dtype)
print(bureau_final['SK_ID_CURR'].dtype)
print(bureau_balance__agg_def_v1['SK_ID_CURR'].dtype)
print(previous_application_agg['SK_ID_CURR'].dtype)


int64
int64
int64
int64


In [27]:
#Uniendo los datos
app_train = app_train.merge(bureau_final, on='SK_ID_CURR', how='left')
app_train = app_train.merge(bureau_balance__agg_def_v1, on='SK_ID_CURR', how='left')
app_train = app_train.merge(previous_application_agg, on='SK_ID_CURR', how='left')
app_train = app_train.merge(pos_cash_agg, on='SK_ID_CURR', how='left')
app_train = app_train.merge(installment_payment_agg, on='SK_ID_CURR', how='left')
app_train = app_train.merge(credit_card_balance, on='SK_ID_CURR', how='left')
######################
# IMPLEMENTACIÓN "A LO BRUTO", NO ES LA MANERA CORRECTO, PERO QUIERO HACER ALGUNAS PRUEBAS. INVESTIGAR MÉTODOS DE HACERLO DE UNA FORMA MÁS EFICIENTE
######################
app_test_pruebav1 = app_test_pruebav1.merge(bureau_final, on='SK_ID_CURR', how='left')
app_test_pruebav1 = app_test_pruebav1.merge(bureau_balance__agg_def_v1, on='SK_ID_CURR', how='left')
app_test_pruebav1 = app_test_pruebav1.merge(previous_application_agg, on='SK_ID_CURR', how='left')
#Para test
app_test_pruebav1['AGE_INT'] = app_test_pruebav1['DAYS_BIRTH']/-365
app_test_pruebav1['HAS_CHILDREN'] = np.where(app_test_pruebav1['CNT_CHILDREN'] > 0, 1, 0)                       

app_train.drop(columns=['SK_ID_CURR'], inplace=True)
#Debido a un fallo que detecta que todas los valores de la columna DAYS_BIRTH 
# son nan cuando no es así, vamos a sustituir la columna por una nueva llamada AGE_INT
app_train['AGE_INT'] = app_train['DAYS_BIRTH']/-365
app_train['HAS_CHILDREN'] = np.where(app_train['CNT_CHILDREN'] > 0, 1, 0)
#variables_categoricas = app_train.select_dtypes(include=['object', 'category']).columns
#app_train[variables_categoricas] = app_train[variables_categoricas].astype('category')
#categoricas = variables_categoricas.tolist()
info = app_train.info()
print(info)
print(app_test_pruebav1.info())
#describe = app_train.describe()
#print(describe)
print(app_test_pruebav1.shape)
print(app_train.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 1120 entries, TARGET to HAS_CHILDREN
dtypes: float64(1060), int64(41), object(19)
memory usage: 2.6+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 908 entries, SK_ID_CURR to HAS_CHILDREN
dtypes: float64(848), int64(41), object(19)
memory usage: 337.7+ MB
None
(48744, 908)
(307511, 1120)


In [63]:
print(app_test_pruebav1['SK_ID_CURR'])
app_train.select_dtypes(include=['int64']).columns 

0        100001
1        100005
2        100013
3        100028
4        100038
          ...  
48739    456221
48740    456222
48741    456223
48742    456224
48743    456250
Name: SK_ID_CURR, Length: 48744, dtype: int64


Index(['TARGET', 'CNT_CHILDREN', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'DAYS_ID_PUBLISH', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE',
       'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'REGION_RATING_CLIENT',
       'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START',
       'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
       'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
       'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2',
       'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5',
       'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8',
       'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
       'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
       'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17',
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'HAS_CHILDREN'],
      dtype='object')

In [28]:
#Feature Engineering


#Vamos a convertir la variable CNT_CHILDREN en una variable categórica que indique 1 si tiene hijos y 0 si no.

#Vamos a crear la variable DAYS_EMPLOYED en una variable categórica que indique 1 si tiene empleo y 0 si no.
app_train['HAS_EMPLOYMENT'] = app_train['DAYS_EMPLOYED'].map(lambda app_train: 1 if app_train < 0 else 0)
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)


#Voy a crear RATIOS para algunas de las variables más importantes (en mi opinión y según lo investigado) 
# y que además tienen valores atípicos.
app_train['CREDIT_AMT_INCOME_RATIO'] = app_train['AMT_CREDIT']/app_train['AMT_INCOME_TOTAL']
app_train['CREDIT_AMT_INCOME_PORCENTAJE'] = app_train['AMT_INCOME_TOTAL']/app_train['AMT_CREDIT']

app_train['ANNUITY_AMT_INCOME_RATIO'] = app_train['AMT_INCOME_TOTAL']/app_train['AMT_ANNUITY']
app_train['ANNUITY_INCOME_PORCENTAJE'] = app_train['AMT_ANNUITY']/app_train['AMT_INCOME_TOTAL']

app_train['GOODS_INCOME_RATIO'] = app_train['AMT_GOODS_PRICE']/app_train['AMT_INCOME_TOTAL']

app_train['CREDIT_ANNUITY_RATIO'] = app_train['AMT_CREDIT']/app_train['AMT_ANNUITY']
app_train['ANNUITY_CREDIT_PORCETAJE'] = app_train['AMT_ANNUITY']/app_train['AMT_CREDIT']
app_train['CREDIT_GOODS_PRICE_RATIO'] = app_train['AMT_CREDIT']/app_train['AMT_GOODS_PRICE']

app_train['ANNUITY_TO_DAYS_EMPLOYED_RATIO'] = app_train['AMT_ANNUITY']/app_train['DAYS_EMPLOYED']
app_train['ANNUITY_TO_CHILDREN_RATIO'] = app_train['AMT_ANNUITY']/app_train['CNT_CHILDREN']

app_train['INCOME_PER_CHILD'] = app_train['AMT_INCOME_TOTAL']/app_train['CNT_CHILDREN']
app_train['CREDIT_PER_CHILD'] = app_train['AMT_CREDIT']/app_train['CNT_CHILDREN']

app_train['AMT_INCOME_FAMILY_MEMBERS_RATIO'] = app_train['AMT_INCOME_TOTAL']/app_train['CNT_FAM_MEMBERS']
app_train['CHILDREN_RATIO'] = app_train['CNT_CHILDREN']/app_train['CNT_FAM_MEMBERS']
app_train['ADULTS_MEMBERS_DIFF'] = app_train['CNT_FAM_MEMBERS'] - app_train['CNT_CHILDREN']

app_train['INCOME_PER_ADULT'] = app_train['AMT_INCOME_TOTAL']/app_train['ADULTS_MEMBERS_DIFF']
app_train['CREDIT_PER_MEMBER'] = app_train['AMT_CREDIT']/app_train['CNT_FAM_MEMBERS']
app_train['CREDIT_PER_ADULT'] = app_train['AMT_CREDIT']/app_train['ADULTS_MEMBERS_DIFF']
app_train['ANNUITY_PER_ADULT'] = app_train['AMT_ANNUITY']/app_train['ADULTS_MEMBERS_DIFF']


#app_train['OWN_CAR_AGE_RATIO'] = app_train['OWN_CAR_AGE']/app_train['AGE_INT']
app_train['NEW_CAR_BIRTH_RATIO'] = app_train['OWN_CAR_AGE']/app_train['DAYS_BIRTH']
app_train['NEW_CAR_EMPLOYMENT_RATIO'] = app_train['OWN_CAR_AGE']/app_train['DAYS_EMPLOYED']


app_train['EMPLOYMENT_AGE_RATIO'] = app_train['DAYS_EMPLOYED']/-app_train['AGE_INT']
#app_train['EMPLOYMENT_BIRTH_RATIO'] = app_train['DAYS_EMPLOYED']/app_train['DAYS_BIRTH']
app_train['EMPLOYMENT_PHONE_CHANGE_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE']/app_train['DAYS_EMPLOYED']

app_train['NEW_PHONE_CHANGE_BIRTH_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE']/app_train['DAYS_BIRTH']
app_train['NEW_PHONE_CHANGE_EMPLOYMENT_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE']/app_train['DAYS_EMPLOYED']
app_train['DAYS_ID_PUBLISH_BIRTH_RAIO'] = app_train['DAYS_ID_PUBLISH']/app_train['DAYS_BIRTH']

app_train['REGISTRATION_TO_NEW_PHONE_CHANGE_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE']/app_train['DAYS_REGISTRATION']
#app_train['REGISTRATION_TO_ID_PUBLISH_RATIO'] = app_train['DAYS_ID_PUBLISH'] - app_train['DAYS_REGISTRATION']
#app_train['REGISTRATION_TO_BIRTH_RATIO'] = app_train['DAYS_BIRTH'] - app_train['DAYS_REGISTRATION']

#EXT_SOURCE son variables representan fuentes externas de información sobre el cliente, y suelen estar relacionadas con puntuaciones de riesgo crediticio generadas por 
# instituciones externas al prestamista, como burós de crédito u otras entidades que evalúan el perfil financiero de los clientes.
#Vamos a crear variables relacionadas con EXT_SOURCE.

app_train['EXT_SOURCE_SUM'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
app_train['EXT_SOURCE_MEAN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
app_train['EXT_SOURCE_MEDIAN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1)
app_train['EXT_SOURCE_MAX'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)
app_train['EXT_SOURCE_MIN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_train['EXT_SOURCE_WEIGHTED_SUM'] = app_train['EXT_SOURCE_1']*3 + app_train['EXT_SOURCE_2']*1 + app_train['EXT_SOURCE_3']*5
app_train['EXT_SOURCE_WEIGHTED_MEAN'] = (app_train['EXT_SOURCE_1']*3 + app_train['EXT_SOURCE_2']*1 + app_train['EXT_SOURCE_3']*5)/3
app_train['EXT_SOURCE_PROD'] = app_train['EXT_SOURCE_1']*app_train['EXT_SOURCE_2']*app_train['EXT_SOURCE_3']

app_train['RATIO_EXT_SOURCE_3_TO_REGION_POPULATION_RELATIVE'] = app_train['EXT_SOURCE_3'] / app_train['REGION_POPULATION_RELATIVE']

"""
app_train['AGE_CREDIT_RATIO'] = app_train['AGE_INT']/app_train['AMT_CREDIT']
app_train['AGE_ANNUITY_RATIO'] = app_train['AGE_INT']/app_train['AMT_ANNUITY']
app_train['AGE_GOODS_PRICE_RATIO'] = app_train['AGE_INT']/app_train['AMT_GOODS_PRICE']
app_train['AGE_INCOME_RATIO'] = app_train['AGE_INT']/app_train['AMT_INCOME_TOTAL']
"""
app_train['AGE_CREDIT_RATIO'] = app_train['AMT_CREDIT']/app_train['DAYS_BIRTH']
app_train['AGE_ANNUITY_RATIO'] = app_train['AMT_ANNUITY']/app_train['DAYS_BIRTH']
app_train['AGE_GOODS_PRICE_RATIO'] = app_train['AMT_GOODS_PRICE']/app_train['DAYS_BIRTH']
app_train['AGE_INCOME_RATIO'] = app_train['AMT_INCOME_TOTAL']/app_train['DAYS_BIRTH']



#Variables a partir de sumas y diferencias
app_train['AMT_INCOME_TOTAL_ANNUITY_SUM'] = app_train['AMT_INCOME_TOTAL'] + app_train['AMT_ANNUITY']
app_train['AMT_GOODS_TO_INCOME_ANUITY_SUM_RATIO'] = app_train['AMT_GOODS_PRICE']/(app_train['AMT_INCOME_TOTAL_ANNUITY_SUM'] )
app_train['CREDIT_BUREAU_TOTAL'] = app_train[['AMT_REQ_CREDIT_BUREAU_DAY', 
                                             'AMT_REQ_CREDIT_BUREAU_HOUR',
                                             'AMT_REQ_CREDIT_BUREAU_MON',
                                             'AMT_REQ_CREDIT_BUREAU_QRT',
                                             'AMT_REQ_CREDIT_BUREAU_WEEK',
                                             'AMT_REQ_CREDIT_BUREAU_YEAR']].sum(axis=1)

app_train['CREDIT_GOODS_DIFF'] = app_train['AMT_GOODS_PRICE'] - app_train['AMT_CREDIT']
app_train['GOODS_ANNUITY_DIFF'] = app_train['AMT_ANNUITY'] - app_train['AMT_GOODS_PRICE']
app_train['AMT_INCOME_TOTAL_ANNUITY_DIFF'] = app_train['AMT_INCOME_TOTAL'] - app_train['AMT_ANNUITY']

#app_train['SOCIAL_OBSERVATION_TOTAL'] = app_train['OBS_30_CNT_SOCIAL_CIRCLE'] + app_train['OBS_60_CNT_SOCIAL_CIRCLE']
#app_train['SOCIAL_DEF_TOTAL'] = app_train['DEF_30_CNT_SOCIAL_CIRCLE'] + app_train['DEF_60_CNT_SOCIAL_CIRCLE']

app_train['DIFF_OBS_30_60'] = app_train['OBS_30_CNT_SOCIAL_CIRCLE'] - app_train['OBS_60_CNT_SOCIAL_CIRCLE']
app_train['DIFF_DEF_30_60'] = app_train['DEF_30_CNT_SOCIAL_CIRCLE'] - app_train['DEF_60_CNT_SOCIAL_CIRCLE'] 

app_train['LONG_EMPLOYMENT'] = np.where(app_train['DAYS_EMPLOYED'] < -2000, 1, 0)
app_train['RETIRED'] = np.where(app_train['DAYS_BIRTH'] < -14000, 1, 0)

#OTRAS






In [65]:
# -- Feature Engineering para app_test_pruebav1 --

# Vamos a convertir la variable CNT_CHILDREN en una variable categórica que indique 1 si tiene hijos y 0 si no.
# (Ya hiciste algo parecido en la creación de la columna HAS_CHILDREN, 
#  si lo necesitas nuevamente, lo mantienes. De lo contrario, omítelo.)

# Vamos a crear la variable DAYS_EMPLOYED en una variable categórica que indique 1 si tiene empleo y 0 si no.
app_test_pruebav1['HAS_EMPLOYMENT'] = app_test_pruebav1['DAYS_EMPLOYED'].map(lambda x: 1 if x < 0 else 0)
app_test_pruebav1['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)

# Creación de RATIOS
app_test_pruebav1['CREDIT_AMT_INCOME_RATIO'] = app_test_pruebav1['AMT_CREDIT'] / app_test_pruebav1['AMT_INCOME_TOTAL']
app_test_pruebav1['CREDIT_AMT_INCOME_PORCENTAJE'] = app_test_pruebav1['AMT_INCOME_TOTAL'] / app_test_pruebav1['AMT_CREDIT']

app_test_pruebav1['ANNUITY_AMT_INCOME_RATIO'] = app_test_pruebav1['AMT_INCOME_TOTAL'] / app_test_pruebav1['AMT_ANNUITY']
app_test_pruebav1['ANNUITY_INCOME_PORCENTAJE'] = app_test_pruebav1['AMT_ANNUITY'] / app_test_pruebav1['AMT_INCOME_TOTAL']

app_test_pruebav1['GOODS_INCOME_RATIO'] = app_test_pruebav1['AMT_GOODS_PRICE'] / app_test_pruebav1['AMT_INCOME_TOTAL']

app_test_pruebav1['CREDIT_ANNUITY_RATIO'] = app_test_pruebav1['AMT_CREDIT'] / app_test_pruebav1['AMT_ANNUITY']
app_test_pruebav1['ANNUITY_CREDIT_PORCETAJE'] = app_test_pruebav1['AMT_ANNUITY'] / app_test_pruebav1['AMT_CREDIT']
app_test_pruebav1['CREDIT_GOODS_PRICE_RATIO'] = app_test_pruebav1['AMT_CREDIT'] / app_test_pruebav1['AMT_GOODS_PRICE']

app_test_pruebav1['ANNUITY_TO_DAYS_EMPLOYED_RATIO'] = app_test_pruebav1['AMT_ANNUITY'] / app_test_pruebav1['DAYS_EMPLOYED']
app_test_pruebav1['ANNUITY_TO_CHILDREN_RATIO'] = app_test_pruebav1['AMT_ANNUITY'] / app_test_pruebav1['CNT_CHILDREN']

app_test_pruebav1['INCOME_PER_CHILD'] = app_test_pruebav1['AMT_INCOME_TOTAL'] / app_test_pruebav1['CNT_CHILDREN']
app_test_pruebav1['CREDIT_PER_CHILD'] = app_test_pruebav1['AMT_CREDIT'] / app_test_pruebav1['CNT_CHILDREN']

app_test_pruebav1['AMT_INCOME_FAMILY_MEMBERS_RATIO'] = app_test_pruebav1['AMT_INCOME_TOTAL'] / app_test_pruebav1['CNT_FAM_MEMBERS']
app_test_pruebav1['CHILDREN_RATIO'] = app_test_pruebav1['CNT_CHILDREN'] / app_test_pruebav1['CNT_FAM_MEMBERS']
app_test_pruebav1['ADULTS_MEMBERS_DIFF'] = app_test_pruebav1['CNT_FAM_MEMBERS'] - app_test_pruebav1['CNT_CHILDREN']

app_test_pruebav1['INCOME_PER_ADULT'] = app_test_pruebav1['AMT_INCOME_TOTAL'] / app_test_pruebav1['ADULTS_MEMBERS_DIFF']
app_test_pruebav1['CREDIT_PER_MEMBER'] = app_test_pruebav1['AMT_CREDIT'] / app_test_pruebav1['CNT_FAM_MEMBERS']
app_test_pruebav1['CREDIT_PER_ADULT'] = app_test_pruebav1['AMT_CREDIT'] / app_test_pruebav1['ADULTS_MEMBERS_DIFF']
app_test_pruebav1['ANNUITY_PER_ADULT'] = app_test_pruebav1['AMT_ANNUITY'] / app_test_pruebav1['ADULTS_MEMBERS_DIFF']

# app_test_pruebav1['OWN_CAR_AGE_RATIO'] = app_test_pruebav1['OWN_CAR_AGE'] / app_test_pruebav1['AGE_INT']
app_test_pruebav1['NEW_CAR_BIRTH_RATIO'] = app_test_pruebav1['OWN_CAR_AGE'] / app_test_pruebav1['DAYS_BIRTH']
app_test_pruebav1['NEW_CAR_EMPLOYMENT_RATIO'] = app_test_pruebav1['OWN_CAR_AGE'] / app_test_pruebav1['DAYS_EMPLOYED']

app_test_pruebav1['EMPLOYMENT_AGE_RATIO'] = app_test_pruebav1['DAYS_EMPLOYED'] / -app_test_pruebav1['AGE_INT']
#app_test_pruebav1['EMPLOYMENT_BIRTH_RATIO'] = app_test_pruebav1['DAYS_EMPLOYED'] / app_test_pruebav1['DAYS_BIRTH']
app_test_pruebav1['EMPLOYMENT_PHONE_CHANGE_RATIO'] = app_test_pruebav1['DAYS_LAST_PHONE_CHANGE'] / app_test_pruebav1['DAYS_EMPLOYED']

app_test_pruebav1['NEW_PHONE_CHANGE_BIRTH_RATIO'] = app_test_pruebav1['DAYS_LAST_PHONE_CHANGE'] / app_test_pruebav1['DAYS_BIRTH']
app_test_pruebav1['NEW_PHONE_CHANGE_EMPLOYMENT_RATIO'] = app_test_pruebav1['DAYS_LAST_PHONE_CHANGE'] / app_test_pruebav1['DAYS_EMPLOYED']
app_test_pruebav1['DAYS_ID_PUBLISH_BIRTH_RAIO'] = app_test_pruebav1['DAYS_ID_PUBLISH'] / app_test_pruebav1['DAYS_BIRTH']

app_test_pruebav1['REGISTRATION_TO_NEW_PHONE_CHANGE_RATIO'] = app_test_pruebav1['DAYS_LAST_PHONE_CHANGE'] / app_test_pruebav1['DAYS_REGISTRATION']
#app_test_pruebav1['REGISTRATION_TO_ID_PUBLISH_RATIO'] = app_test_pruebav1['DAYS_ID_PUBLISH'] - app_test_pruebav1['DAYS_REGISTRATION']
#app_test_pruebav1['REGISTRATION_TO_BIRTH_RATIO'] = app_test_pruebav1['DAYS_BIRTH'] - app_test_pruebav1['DAYS_REGISTRATION']

#EXT_SOURCE => creamos variables relacionadas con EXT_SOURCE_1,2,3
app_test_pruebav1['EXT_SOURCE_SUM'] = app_test_pruebav1[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
app_test_pruebav1['EXT_SOURCE_MEAN'] = app_test_pruebav1[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
app_test_pruebav1['EXT_SOURCE_MEDIAN'] = app_test_pruebav1[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1)
app_test_pruebav1['EXT_SOURCE_MAX'] = app_test_pruebav1[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)
app_test_pruebav1['EXT_SOURCE_MIN'] = app_test_pruebav1[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_test_pruebav1['EXT_SOURCE_WEIGHTED_SUM'] = (
    app_test_pruebav1['EXT_SOURCE_1'] * 3
  + app_test_pruebav1['EXT_SOURCE_2'] * 1
  + app_test_pruebav1['EXT_SOURCE_3'] * 5
)
app_test_pruebav1['EXT_SOURCE_WEIGHTED_MEAN'] = (
    (app_test_pruebav1['EXT_SOURCE_1'] * 3
    + app_test_pruebav1['EXT_SOURCE_2'] * 1
    + app_test_pruebav1['EXT_SOURCE_3'] * 5) / 3
)
app_test_pruebav1['EXT_SOURCE_PROD'] = (
    app_test_pruebav1['EXT_SOURCE_1']
  * app_test_pruebav1['EXT_SOURCE_2']
  * app_test_pruebav1['EXT_SOURCE_3']
)

app_test_pruebav1['RATIO_EXT_SOURCE_3_TO_REGION_POPULATION_RELATIVE'] = (
    app_test_pruebav1['EXT_SOURCE_3'] / app_test_pruebav1['REGION_POPULATION_RELATIVE']
)

# Ejemplo en comentarios: la inversa
app_test_pruebav1['AGE_CREDIT_RATIO'] = app_test_pruebav1['AMT_CREDIT'] / app_test_pruebav1['DAYS_BIRTH']
app_test_pruebav1['AGE_ANNUITY_RATIO'] = app_test_pruebav1['AMT_ANNUITY'] / app_test_pruebav1['DAYS_BIRTH']
app_test_pruebav1['AGE_GOODS_PRICE_RATIO'] = app_test_pruebav1['AMT_GOODS_PRICE'] / app_test_pruebav1['DAYS_BIRTH']
app_test_pruebav1['AGE_INCOME_RATIO'] = app_test_pruebav1['AMT_INCOME_TOTAL'] / app_test_pruebav1['DAYS_BIRTH']

#Variables a partir de sumas y diferencias
app_test_pruebav1['AMT_INCOME_TOTAL_ANNUITY_SUM'] = (
    app_test_pruebav1['AMT_INCOME_TOTAL'] + app_test_pruebav1['AMT_ANNUITY']
)
app_test_pruebav1['AMT_GOODS_TO_INCOME_ANUITY_SUM_RATIO'] = (
    app_test_pruebav1['AMT_GOODS_PRICE'] / app_test_pruebav1['AMT_INCOME_TOTAL_ANNUITY_SUM']
)
app_test_pruebav1['CREDIT_BUREAU_TOTAL'] = app_test_pruebav1[[
    'AMT_REQ_CREDIT_BUREAU_DAY',
    'AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_MON',
    'AMT_REQ_CREDIT_BUREAU_QRT',
    'AMT_REQ_CREDIT_BUREAU_WEEK',
    'AMT_REQ_CREDIT_BUREAU_YEAR'
]].sum(axis=1)

app_test_pruebav1['CREDIT_GOODS_DIFF'] = app_test_pruebav1['AMT_GOODS_PRICE'] - app_test_pruebav1['AMT_CREDIT']
app_test_pruebav1['GOODS_ANNUITY_DIFF'] = app_test_pruebav1['AMT_ANNUITY'] - app_test_pruebav1['AMT_GOODS_PRICE']
app_test_pruebav1['AMT_INCOME_TOTAL_ANNUITY_DIFF'] = (
    app_test_pruebav1['AMT_INCOME_TOTAL'] - app_test_pruebav1['AMT_ANNUITY']
)

# app_test_pruebav1['SOCIAL_OBSERVATION_TOTAL'] = ...
# app_test_pruebav1['SOCIAL_DEF_TOTAL'] = ...

app_test_pruebav1['DIFF_OBS_30_60'] = (
    app_test_pruebav1['OBS_30_CNT_SOCIAL_CIRCLE'] - app_test_pruebav1['OBS_60_CNT_SOCIAL_CIRCLE']
)
app_test_pruebav1['DIFF_DEF_30_60'] = (
    app_test_pruebav1['DEF_30_CNT_SOCIAL_CIRCLE'] - app_test_pruebav1['DEF_60_CNT_SOCIAL_CIRCLE']
)

app_test_pruebav1['LONG_EMPLOYMENT'] = np.where(app_test_pruebav1['DAYS_EMPLOYED'] < -2000, 1, 0)
app_test_pruebav1['RETIRED'] = np.where(app_test_pruebav1['DAYS_BIRTH'] < -14000, 1, 0)

# (Si no hay más)
# Fin de las "OTRAS"


In [66]:
print(app_test_pruebav1['SK_ID_CURR'])

0        100001
1        100005
2        100013
3        100028
4        100038
          ...  
48739    456221
48740    456222
48741    456223
48742    456224
48743    456250
Name: SK_ID_CURR, Length: 48744, dtype: int64


In [29]:
education_mapping = {
    'No formal education': 0,
    'Lower secondary': 1,
    'Secondary / secondary special': 2,
    'Incomplete higher': 3,
    'Higher education': 4,
    'Academic degree': 5
}

app_train['EDUCATION_LEVEL'] = app_train['NAME_EDUCATION_TYPE'].map(education_mapping)
app_test_pruebav1['EDUCATION_LEVEL'] = app_test_pruebav1['NAME_EDUCATION_TYPE'].map(education_mapping)
print(app_train['EDUCATION_LEVEL'].describe())
"""
income_type_mapping = {
    'Unemployed': 0,
    'Student': 1,
    'Pensioner': 2,
    'Maternity leave': 3,
    'Working': 4,
    'State servant': 5,
    'Commercial associate': 6,
    'Businessman': 7,    
}
app_train['INCOME_TYPE'] = app_train['NAME_INCOME_TYPE'].map(income_type_mapping)

family_mapping = {
    'Single / not married': 0,
    'Separated': 1,
    'Widow': 2,
    'Married': 3
}

housing_mapping = {
    'With parents': 0,
    'Municipal apartment': 1,
    'Rented apartment': 2,
    'House / apartment': 3
}



app_train['FAMILY_STATUS'] = app_train['NAME_FAMILY_STATUS'].map(family_mapping)
app_train['HOUSING_TYPE'] = app_train['NAME_HOUSING_TYPE'].map(housing_mapping)

"""

count    307511.000000
mean          2.509507
std           0.874544
min           1.000000
25%           2.000000
50%           2.000000
75%           3.000000
max           5.000000
Name: EDUCATION_LEVEL, Length: 8, dtype: float64


"\nincome_type_mapping = {\n    'Unemployed': 0,\n    'Student': 1,\n    'Pensioner': 2,\n    'Maternity leave': 3,\n    'Working': 4,\n    'State servant': 5,\n    'Commercial associate': 6,\n    'Businessman': 7,    \n}\napp_train['INCOME_TYPE'] = app_train['NAME_INCOME_TYPE'].map(income_type_mapping)\n\nfamily_mapping = {\n    'Single / not married': 0,\n    'Separated': 1,\n    'Widow': 2,\n    'Married': 3\n}\n\nhousing_mapping = {\n    'With parents': 0,\n    'Municipal apartment': 1,\n    'Rented apartment': 2,\n    'House / apartment': 3\n}\n\n\n\napp_train['FAMILY_STATUS'] = app_train['NAME_FAMILY_STATUS'].map(family_mapping)\napp_train['HOUSING_TYPE'] = app_train['NAME_HOUSING_TYPE'].map(housing_mapping)\n\n"

In [30]:
#AGREGACIONES
agregaciones = [
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max'),
                                              ('AMT_CREDIT', 'max'),
                                              ('EXT_SOURCE_1', 'mean'),
                                              ('EXT_SOURCE_2', 'mean'),
                                              ('EXT_SOURCE_3', 'mean'),
                                              ('AGE_INT', 'mean'),
                                              ('OWN_CAR_AGE', 'max')]),

    (['CODE_GENDER', 'ORGANIZATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                                            ('AMT_CREDIT', 'mean'),
                                            ('EXT_SOURCE_1', 'mean'),
                                            ('EXT_SOURCE_2', 'mean'),
                                            ('EXT_SOURCE_3', 'mean')]),
    (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [('AMT_ANNUITY', 'mean'),
                                                ('DAYS_LAST_PHONE_CHANGE', 'mean'),
                                                ('DAYS_REGISTRATION', 'mean'),
                                                ('CNT_CHILDREN', 'mean')]),

                                              
                                                                                                                                             
]
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(agregaciones):
    group_object = app_train.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        app_train = app_train.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [69]:
#AGREGACIONES
agregaciones = [
    (['CODE_GENDER', 'NAME_EDUCATION_TYPE'], [('AMT_ANNUITY', 'max'),
                                              ('AMT_CREDIT', 'max'),
                                              ('EXT_SOURCE_1', 'mean'),
                                              ('EXT_SOURCE_2', 'mean'),
                                              ('EXT_SOURCE_3', 'mean'),
                                              ('AGE_INT', 'mean'),
                                              ('OWN_CAR_AGE', 'max')]),

    (['CODE_GENDER', 'ORGANIZATION_TYPE'], [('AMT_ANNUITY', 'mean'),
                                            ('AMT_CREDIT', 'mean'),
                                            ('EXT_SOURCE_1', 'mean'),
                                            ('EXT_SOURCE_2', 'mean'),
                                            ('EXT_SOURCE_3', 'mean')]),
    (['CODE_GENDER', 'REG_CITY_NOT_WORK_CITY'], [('AMT_ANNUITY', 'mean'),
                                                ('DAYS_LAST_PHONE_CHANGE', 'mean'),
                                                ('DAYS_REGISTRATION', 'mean'),
                                                ('CNT_CHILDREN', 'mean')]),

                                              
                                                                                                                                             
]
groupby_aggregate_names = []
for groupby_cols, specs in tqdm(agregaciones):
    group_object = app_test_pruebav1.groupby(groupby_cols)
    for select, agg in tqdm(specs):
        groupby_aggregate_name = '{}_{}_{}'.format('_'.join(groupby_cols), agg, select)
        app_test_pruebav1 = app_test_pruebav1.merge(group_object[select]
                              .agg(agg)
                              .reset_index()
                              .rename(index=str,
                                      columns={select: groupby_aggregate_name})
                              [groupby_cols + [groupby_aggregate_name]],
                              on=groupby_cols,
                              how='left')
        groupby_aggregate_names.append(groupby_aggregate_name)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [70]:
all_nan_cols = [i for i in app_test_pruebav1 if app_test_pruebav1[i].isnull().all()]
print(all_nan_cols)
#app_test_pruebav1.drop(columns=all_nan_cols, inplace=True)
print(app_test_pruebav1.info())
print(app_test_pruebav1['SK_ID_CURR'])

[]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 976 entries, SK_ID_CURR to CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN
dtypes: float64(913), int64(44), object(19)
memory usage: 363.0+ MB
None
0        100001
1        100005
2        100013
3        100028
4        100038
          ...  
48739    456221
48740    456222
48741    456223
48742    456224
48743    456250
Name: SK_ID_CURR, Length: 48744, dtype: int64


In [31]:
# Preprocesamiento previo a la creación de los modelos
# Imputación de valores nulos. Vamos a probar de momento a completar variables numéricos con la media
# y las categóricas con la moda.
# Como sabemos que DAYS_EMPLOYEED tiene un valor erroneo, vamos a reemplazarlo por NaN
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
app_train['DAYS_LAST_PHONE_CHANGE'].replace({0: np.nan}, inplace = True)
app_train['NAME_FAMILY_STATUS'].replace('Unknown', np.nan, inplace=True)
app_train.replace([np.inf, -np.inf], np.nan, inplace=True)

app_test_pruebav1['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
app_test_pruebav1['DAYS_LAST_PHONE_CHANGE'].replace({0: np.nan}, inplace = True)
app_test_pruebav1['NAME_FAMILY_STATUS'].replace('Unknown', np.nan, inplace=True)
app_test_pruebav1.replace([np.inf, -np.inf], np.nan, inplace=True)
# Algunas variables categóricas tienen valores como XNA, vamos a reemplazarlos por NaN
def reemplazar_xna_por_nan(app_train):
    variables_categoricas = app_train.select_dtypes(include=['object', 'category']).columns
    for col in variables_categoricas:
        app_train[col] = app_train[col].replace('XNA', np.nan)
        #app_train[col] = app_train[col].replace('XAP', np.nan)
    return app_train
app_train_noxna = reemplazar_xna_por_nan(app_train)
app_test_noxna = reemplazar_xna_por_nan(app_test_pruebav1)

def imputar_valores_nulos(app_train):
    variables_continuas = app_train.select_dtypes(include=[np.number]).columns 
    for col in variables_continuas:
        app_train[col] = app_train[col].fillna(app_train[col].mean())
    variables_categoricas = app_train.select_dtypes(include=['object', 'category']).columns
    for col in variables_categoricas:
        app_train[col] = app_train[col].fillna(app_train[col].mode()[0])

    return app_train

app_train_nonan = imputar_valores_nulos(app_train_noxna)
app_test_nonan = imputar_valores_nulos(app_test_noxna)

#Comprobamos que no haya valores nulos
#print(app_train['DAYS_BIRTH'].unique())
#print(app_train['DAYS_BIRTH'].isnull().sum())
#print(app_train['DAYS_BIRTH'].dtype)

# Verificar si hay valores nulos en app_train después de la imputación
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan.isnull().sum().sum())
print("Valores nulos en app_test después de la imputación:")
print(app_test_nonan.isnull().sum().sum())
print(app_test_nonan['SK_ID_CURR'])

Valores nulos en app_train después de la imputación:
0
Valores nulos en app_test después de la imputación:
146232
0        100001
1        100005
2        100013
3        100028
4        100038
          ...  
48739    456221
48740    456222
48741    456223
48742    456224
48743    456250
Name: SK_ID_CURR, Length: 48744, dtype: int64


In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

label_encoders_binarias = {}

def label_encoding_binarias_fit(df):
    variables_categoricas = df.select_dtypes(include=['object','category']).columns
    variables_categoricas_binarias = [col for col in variables_categoricas if df[col].nunique() == 2]
    for col in variables_categoricas_binarias:
        enc = LabelEncoder()
        enc.fit(df[col])
        label_encoders_binarias[col] = enc

def label_encoding_binarias_transform(df):
    df = df.copy()
    for col in label_encoders_binarias:
        df[col] = label_encoders_binarias[col].transform(df[col])
    return df

def one_hot_encoding_no_binarias(df, df_train):
    df = df.copy()
    variables_categoricas = df_train.select_dtypes(include=['object','category']).columns
    variables_categoricas_no_binarias = [col for col in variables_categoricas if df_train[col].nunique() > 2]
    df = pd.get_dummies(df, columns=variables_categoricas_no_binarias)
    return df

app_train_noxna = app_train_noxna.select_dtypes(include=[np.number, 'object','category'])
app_test_noxna = app_test_noxna.select_dtypes(include=[np.number, 'object','category'])

label_encoding_binarias_fit(app_train_noxna)
app_train_nonan_le = label_encoding_binarias_transform(app_train_noxna)
app_test_nonan_le = label_encoding_binarias_transform(app_test_noxna)

app_train_nonan_le_oh = one_hot_encoding_no_binarias(app_train_nonan_le, app_train_nonan_le)
app_test_nonan_le_oh = one_hot_encoding_no_binarias(app_test_nonan_le, app_train_nonan_le)

#common_cols = app_train_nonan_le_oh.columns
#app_test_nonan_le_oh = app_test_nonan_le_oh.reindex(columns=common_cols, fill_value=0)

print(app_train_nonan_le_oh.shape, app_test_nonan_le_oh.shape)
print("Valores nulos en app_train después de la imputación:")
print(app_train_nonan_le_oh.isnull().sum().sum())
print("Valores nulos en app_test después de la imputación:")
print(app_test_nonan_le_oh.isnull().sum().sum())
print(app_test_nonan_le_oh['SK_ID_CURR'])

(307511, 1322) (48744, 1038)
Valores nulos en app_train después de la imputación:
0
Valores nulos en app_test después de la imputación:
146232
0        100001
1        100005
2        100013
3        100028
4        100038
          ...  
48739    456221
48740    456222
48741    456223
48742    456224
48743    456250
Name: SK_ID_CURR, Length: 48744, dtype: int64


In [33]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
import pandas as pd

# Identificar las variables continuas que necesitan normalización
variables_continuas = app_train_nonan_le_oh.select_dtypes(include=['int64', 'float64']).columns
variables_normalizadas = [col for col in variables_continuas if
                          app_train_nonan_le_oh[col].min() >= 0 and app_train_nonan_le_oh[col].max() <= 1]
variables_NO_normalizadas = [col for col in variables_continuas if col not in variables_normalizadas]


variables_continuas_test = app_test_nonan_le_oh.select_dtypes(include=['int64', 'float64']).columns
variables_normalizadas_test = [col for col in variables_continuas_test if
                          app_test_nonan_le_oh[col].min() >= 0 and app_test_nonan_le_oh[col].max() <= 1]
variables_NO_normalizadas_test = [col for col in variables_continuas_test if col not in variables_normalizadas_test]

# Diccionario para almacenar los escaladores ajustados
scalers = {}

# Función para ajustar el escalador a las variables no normalizadas
def normalizacion_fit(data, cols, scaler_type='minmax'):
    for col in cols:
        if scaler_type == 'minmax':
            scaler = MinMaxScaler()
        elif scaler_type == 'robust':
            scaler = RobustScaler()
        else:
            raise ValueError("Tipo de escalador no reconocido. Usa 'minmax' o 'robust'.")
        scaler.fit(data[[col]])
        scalers[col] = scaler

# Función para transformar las variables utilizando los escaladores ajustados
def normalizacion_transform(data, cols):
    data = data.copy()  # Evitar modificar el original
    for col in cols:
        if col in scalers:
            data[col] = scalers[col].transform(data[[col]])
    return data

# Ajustar los escaladores a las variables no normalizadas en el conjunto de entrenamiento
normalizacion_fit(app_train_nonan_le_oh, variables_NO_normalizadas, scaler_type='minmax')
normalizacion_fit(app_test_nonan_le_oh, variables_NO_normalizadas_test, scaler_type='minmax')
# Transformar el conjunto de entrenamiento
app_train_nonan_le_oh_norm = normalizacion_transform(app_train_nonan_le_oh, variables_NO_normalizadas)
app_test_nonan_le_oh_norm = normalizacion_transform(app_test_nonan_le_oh, variables_NO_normalizadas_test)

# Verificar los valores nulos y la información del DataFrame transformado
print("Valores nulos en app_train después de la imputación:")
#print(app_train_nonan_le_oh_norm.isnull().sum().sum())
#print(app_train_nonan_le_oh_norm.info())
#print(app_train_nonan_le_oh_norm.max())
print(app_test_nonan_le_oh_norm.isnull().sum().sum())
print(app_test_nonan_le_oh_norm.info())
print(app_test_nonan_le_oh_norm.max())

Valores nulos en app_train después de la imputación:
146232
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 1038 entries, SK_ID_CURR to LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Sold
dtypes: bool(143), float64(857), int64(38)
memory usage: 339.5 MB
None
SK_ID_CURR                                               1.0
NAME_CONTRACT_TYPE                                         1
CODE_GENDER                                                1
FLAG_OWN_CAR                                               1
FLAG_OWN_REALTY                                            1
                                                        ... 
LATEST_CREDIT_TYPE_CAT_(BUREAU)_Unknown type of loan    True
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Active                True
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Bad debt              True
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Closed                True
LATEST_CREDIT_ACTIVE_CAT_(BUREAU)_Sold                  True
Length: 1038, dtype: object


In [34]:
#print(app_train_nonan_le_oh_norm.shape)
print(app_test_nonan_le_oh_norm.shape)
print(app_train_nonan_le_oh_norm.shape)
print(app_test_nonan_le_oh_norm['SK_ID_CURR'])

(48744, 1038)
(307511, 1322)
0        0.000000
1        0.000011
2        0.000034
3        0.000076
4        0.000104
           ...   
48739    0.999919
48740    0.999921
48741    0.999924
48742    0.999927
48743    1.000000
Name: SK_ID_CURR, Length: 48744, dtype: float64


In [35]:
#Exportamos el csv tras todo el preprocesamiento
#TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/bureau_agg_final.csv
#bureau_agg.to_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/bureau_agg_final.csv', index=False)
app_train_nonan_le_oh_norm.to_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v7.csv')
#app_test_nonan_le_oh_norm.to_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/app_test_pruebav1.csv')